# question 1

In [191]:
concrete = pd.read_csv("drive/MyDrive/DATA301_Data/concrete.csv")

In [192]:
y = concrete['strength'].values
x = concrete.loc[:,'cement':'age'].values

In [194]:
from sklearn.model_selection import train_test_split as tts
xtrain, xtest, ytrain, ytest = tts(x,y,test_size =0.3, shuffle=True, random_state =123)


In [195]:
scale = StandardScaler()

In [197]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization # analog of standard scalar
from keras.layers import Dense, Dropout, Activation, Flatten

In [198]:
def makemodel(lr=.01):
  model = Sequential()
  model.add(Dense(16, input_dim=8, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(8, activation='relu'))
  model.add(Dense(8, activation='linear'))
  model.add(Dropout(0.1))
  model.add(Dense(4, activation='gelu'))
  model.add(Flatten())
  model.add(Dense(2, activation='linear'))
  model.add(Dense(1, activation='gelu'))
  model.compile(loss='mse', optimizer=Adam(learning_rate=lr))
  return model

In [200]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error as mse
kf = KFold(n_splits=5,shuffle=True)
mse_tst = []
for idxtrain, idxtest in kf.split(x):
  xtrain = scale.fit_transform(x[idxtrain])
  xtest = scale.transform(x[idxtest])
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  model = makemodel()
  model.fit(xtrain, ytrain, epochs=1000,batch_size=30, verbose=0)
  mse_tst.append(mse(ytest,model.predict(xtest)))
mse_tst
np.mean(mse_tst)

7/7 [==============================] - 0s 4ms/step


35.22279682214855

# question 2

In [37]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.layers import LeakyReLU, ReLU
from keras.regularizers import l1, l2, l1_l2
import pandas as pd

In [38]:
data = pd.read_csv('/content/drive/MyDrive/DATA301_Data/UCI_Credit_Card.csv')

In [39]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [40]:
x = data.loc[:,'LIMIT_BAL':'PAY_AMT6'].values
y = data['default.payment.next.month'].values

In [41]:
xdim = x.shape[1]

In [174]:
model = Sequential()
model.add(Dense(8,activation=LeakyReLU(alpha=0.01),input_dim = xdim))# the first layer
model.add(Dense(18,activation=ReLU(), kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=123)))# the second layer
model.add(Dense(1,kernel_regularizer=l2(0.01),activation='sigmoid')) # the output layer

In [175]:
opt = Adam(learning_rate=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics='accuracy')

In [176]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
xscale = scale.fit_transform(x)

In [177]:
model.fit(xscale,y,batch_size=64,epochs=100,verbose=0)

In [178]:
model.evaluate(xscale,y)

938/938 [==============================] - 3s 3ms/step - loss: 0.4534 - accuracy: 0.8204


[0.4534112811088562, 0.8204333186149597]

In [179]:
proba = model.predict(xscale)

938/938 [==============================] - 2s 2ms/step


In [180]:
y_predicted = (model.predict(xscale) > 0.5).astype("int32").reshape(-1,1)

938/938 [==============================] - 1s 1ms/step


In [181]:
from sklearn.metrics import confusion_matrix

In [182]:
pd.DataFrame(data=confusion_matrix(y,y_predicted), columns = ['0','1'],index = ['0','1'])

,0,1
0,22226,1138
1,4249,2387


In [183]:
from sklearn.model_selection import StratifiedKFold

In [184]:
skf = StratifiedKFold(n_splits=5, shuffle=True,random_state=123)

In [185]:
test_acc = []
for idxtrain, idxtest in skf.split(x,y):
  xtrain = x[idxtrain]
  # we should scale properly
  xtrain = scale.fit_transform(xtrain)
  xtest = x[idxtest]
  xtest = scale.transform(xtest)
  # so we got the input features scaled correctly for cross-validation
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  # the design we want to validate
  test_acc.append(model.evaluate(xtest,ytest)[1])

188/188 [==============================] - 0s 2ms/step - loss: 0.4556 - accuracy: 0.8193


In [186]:
test_acc
sum(test_acc)/5

0.8203333377838135

In [187]:
model.evaluate(xscale,y)
proba = model.predict(xscale)


938/938 [==============================] - 2s 2ms/step
